# Gym environments

## Installing gym

In this course, we will mostly address RL environments available in the **OpenAI Gym** framework:

<https://gym.openai.com>

It provides a multitude of RL problems, from simple text-based problems with a few dozens of states (Gridworld, Taxi) to continuous control problems (Cartpole, Pendulum) to Atari games (Breakout, Space Invaders) to complex robotics simulators (Mujoco):

<https://gym.openai.com/envs>

However, `gym` is not maintained by OpenAI anymore since September 2022. We will use instead the `gymnasium` library maintained by the Farama foundation, which will keep on maintaining and improving the library.

<https://gymnasium.farama.org/>

You can install gymnasium and its dependencies using:

```bash
pip install -U gymnasium pygame moviepy swig
pip install "gymnasium[classic_control]"
pip install "gymnasium[box2d]"
```

For this exercise and the following, we will focus on simple environments whose installation is straightforward: toy text, classic control and box2d. More complex environments based on Atari games or the Mujoco physics simulator are described in the last (optional) section of this notebook, as they require additional dependencies. 

On colab, `gym` cannot open graphical windows for visualizing the environments, as it is not possible in the browser. We will see a workaround allowing to produce videos. Running that cell in colab should allow you to run the simplest environments:

In [ ]:
try:
    import google.colab
    IN_COLAB = True
except:
    IN_COLAB = False

if IN_COLAB:
    !pip install -U gymnasium pygame swig
    !pip install -U moviepy==1.0.3
    !pip install gymnasium[box2d]


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import os

import gymnasium as gym
print("gym version:", gym.__version__)

from moviepy.editor import ImageSequenceClip, ipython_display

class GymRecorder(object):
    """
    Simple wrapper over moviepy to generate a .gif with the frames of a gym environment.
    
    The environment must have the render_mode `rgb_array_list`.
    """
    def __init__(self, env):
        self.env = env
        self._frames = []

    def record(self, frames):
        "To be called at the end of an episode."
        for frame in frames:
            self._frames.append(np.array(frame))

    def make_video(self, filename):
        "Generates the gif video."
        directory = os.path.dirname(os.path.abspath(filename))
        if not os.path.exists(directory):
            os.mkdir(directory)
        self.clip = ImageSequenceClip(list(self._frames), fps=self.env.metadata["render_fps"])
        self.clip.write_gif(filename, fps=self.env.metadata["render_fps"], loop=0)
        del self._frames
        self._frames = []

## Interacting with an environment

A gym environment is created using:

```python
env = gym.make('CartPole-v1', render_mode="human")
```

where 'CartPole-v1' should be replaced by the environment you want to interact with. The following cell lists the environments available to you (including the different versions).

In [ ]:
for env in gym.envs.registry.items():
    print(env[0])

The `render_mode` argument defines how you will see the environment:

* `None` (default): allows to train a DRL algorithm without wasting computational resources rendering it.
* `rgb_array_list`: allows to get numpy arrays corresponding to each frame. Will be useful when generating videos.
* `ansi`: string representation of each state. Only available for the "Toy text" environments.
* `human`: graphical window displaying the environment live.

The main interest of gym(nasium) is that all problems have a common interface defined by the class `gym.Env`. There are only three methods that have to be used when interacting with an environment:

* `state, info = env.reset()` restarts the environment and returns an initial state $s_0$.

* `state, reward, terminal, truncated, info = env.step(action)` takes an action $a_t$ and returns:
    * the new state $s_{t+1}$, 
    * the reward $r_{t+1}$, 
    * two boolean flags indicating whether the current state is terminal (won/lost) or truncated (timeout),
    * a dictionary containing additional info for debugging (you can ignore it most of the time).

* `env.render()` displays the current state of the MDP. When the render mode is set to `rgb_array_list` or `human`, it does not even have to called explicitly (since gym 0.25).

With this interface, we can interact with the environment in a standardized way:

* We first create the environment.
* For a fixed number of episodes:
    * We pick an initial state with `reset()`.
    * Until the episode is terminated:
        * We select an action using our RL algorithm or randomly.
        * We take that action (`step()`), observe the new state and the reward.
        * We go into the new state.

The following cell shows how to interact with the CartPole environment using a random policy. Note that it will only work on your computer, not in colab.

In [ ]:
env = gym.make('CartPole-v1', render_mode="human")

for episode in range(10):
    state, info = env.reset()
    done = False
    while not done:
        # Select an action randomly
        action = env.action_space.sample()
        
        # Sample a single transition
        next_state, reward, terminal, truncated, info = env.step(action)
        
        # Go in the next state
        state = next_state

        # End of the episode
        done = terminal or truncated

env.close()

On colab (or whenever you want to record videos of the episodes instead of watching them live), you need to create the environment with the rendering mode `rgb_array_list`. 

You then create a `GymRecorder` object (defined in the first cell of this notebook). 

```python
recorder = GymRecorder(env)
```

At the end of each episode, you tell the recorder to record all frames generated during the episode. The frames returned by `env.render()` are (width, height, 3) numpy arrays which are accumulated by the environment during the episode and flushed when `env.reset()` is called.

```python
recorder.record(env.render())
```

You can then generate a gif at the end of the simulation with:

```python
recorder.make_video('videos/CartPole-v1.gif')
```

Finally, you can render the gif in the notebook by calling **at the very last line of the cell**:

```python
ipython_display('videos/CartPole-v1.gif')
```

In [ ]:
env = gym.make('CartPole-v1', render_mode="rgb_array_list")
recorder = GymRecorder(env)

for episode in range(10):
    state, info = env.reset()

    done = False
    while not done:
        # Select an action randomly
        action = env.action_space.sample()
        
        # Sample a single transition
        next_state, reward, terminal, truncated, info = env.step(action)
        
        # Go in the next state
        state = next_state

        # End of the episode
        done = terminal or truncated

    # Record at the end of the episode    
    recorder.record(env.render())

recorder.make_video('videos/CartPole-v1.gif')
ipython_display('videos/CartPole-v1.gif', autoplay=1, loop=0)

Each environment defines its state space (`env.observation_space`) and action space (`env.action_space`). 

State and action spaces can either be :

* discrete (`gym.spaces.Discrete(nb_states)`), with states being an integer between 0 and `nb_states` -1.

* feature-based (`gym.spaces.Box(low=0, high=255, shape=(SCREEN_HEIGHT, SCREEN_WIDTH, 3))`) for pixel frames.

* continuous. Example for two joints of a robotic arm limited between -180 and 180 degrees:

```python
gym.spaces.Box(-180.0, 180.0, (2, ))
```

You can sample a state or action randomly from these spaces:

In [ ]:
action_space = gym.spaces.Box(-180.0, 180.0, (2, ))
action = action_space.sample()
print(action)



Sampling the action space is particularly useful for exploration. We use it here to perform random (but valid) actions:

```python
action = env.action_space.sample()
```



**Q:** Create a method `random_interaction(env, number_episodes, recorder=None)` that takes as arguments:

* The environment.
* The number of episodes to be performed.
* An optional `GymRecorder` object that may record the frames of the environment if it is not None (`if renderer is not None:`). Otherwise, do not nothing.

The method should return the list of undiscounted returns ($\gamma=1$, i.e. just the sum of rewards obtained during each episode) for all episodes.

**Q:** Use that method to visualize all the available simple environments for a few episodes:

* CartPole-v1
* MountainCar-v0
* Pendulum-v1
* Acrobot-v1
* LunarLander-v2
* BipedalWalker-v3
* CarRacing-v2
* Blackjack-v1
* FrozenLake-v1
* CliffWalking-v0
* Taxi-v3

If you do many episodes (CarRacing or Taxi have very long episodes with a random policy), plot the obtained returns to see how they vary. 

If you managed to install the mujoco and atari dependencies, feel free to visualize them too. 

## Creating your own environment

### Random environment

You can create your own environment using the gym interface:

<https://gymnasium.farama.org/tutorials/environment_creation/>

Here is an example of a dummy environment with discrete states and actions, where the transition probabilities and rewards are completely random:

In [7]:
class RandomEnv(gym.Env):
    "Random discrete environment that does nothing."
    
    metadata = {"render_modes": ["ansi"], "render_fps": 1}

    def __init__(self, nb_states, nb_actions, max_episode_steps=10, render_mode="ansi"):

        self.nb_states = nb_states
        self.nb_actions = nb_actions
        self.max_episode_steps = max_episode_steps
        self.render_mode = render_mode

        # State space, can be discrete or continuous.
        self.observation_space = gym.spaces.Discrete(nb_states)
        
        # Action space, can be discrete or continuous.
        self.action_space = gym.spaces.Discrete(nb_actions)    

        # Reset
        self.reset()


    def reset(self, seed=None, options=None):

        # Re-initialize time
        self.current_step = 0
        
        # Sample one state randomly 
        self.state = self.observation_space.sample()
        
        return self.state, {}

    def step(self, action):

        # Random transition to another state
        self.state = self.observation_space.sample() 
        
        # Random reward
        reward = np.random.uniform(0, 1, 1)[0] 
        
        # Terminate the episode after 10 steps
        terminal = False 
        truncated = False

        self.current_step +=1
        if self.current_step % self.max_episode_steps == 0:
            truncated = True 

        info = {} # No info

        return self.state, reward, terminal, truncated, info


    def render(self):
        if self.render_mode == "ansi":
            description = "Step " + str(self.current_step) + ": state " + str(self.state)
            return description
        return None


The different methods should be quite self-explanatory.

`metadata` defines which render modes are available for this environment (here only the text mode "ansi").

The constructor accepts the size of the state and action spaces as arguments, the duration of the episode and the render mode. 

`reset()` samples an initial state randomly.

`step()` ignores the action, samples a new state and a reward, and truncates an episode after `max_episode_steps`.

`render()` returns a string with the current state.

**Q:** Interact with the random environment for a couple of episodes.

As the mode is `ansi` (text-based), you will need to print the string returned by `render()` after each step:

```python
while not done:

    action = env.action_space.sample()
    
    next_state, reward, terminal, truncated, info = env.step(action)

    print(env.render())
```

### Recycling robot

**Q:** Create a `RecyclingRobot` gym-like environment using last week's exercise.

The parameters `alpha`, `beta`, `r_wait` and `r_search` should be passed to the constructor of the environment and saved as attributes.

The state space is discrete, with two states `high` and `low` which will have indices 0 and 1. The three discrete actions `search`, `wait` and `recharge` have indices 0, 1, and 2.

The initial state of the MDP (`reset()`) should be the high state.

The `step()` should generate transitions according to the dynamics of the MDP. Depending on the current state and the chosen action, make a transition to another state. For the actions `search` and `wait`, sample the reward from the normal distribution with mean `r_search` (resp. `r_wait`) and variance 0.5. 

If the random agent selects `recharge` in `high`, do nothing (next state is high, reward is 0).

Rendering is just printing the current state. There is nothing to close, so you do not even need to redefine the function.

Although the recycling robot is a continuing task, limit the number of steps per episode to 10, as in the the previous random environment.

Interact randomly with the MDP for several episodes and observe the returns. 

### Random agent

To be complete, let's implement the random agent as a class. The class should look like:

```python
class RandomAgent:
    """
    Random agent exploring uniformly the environment.
    """
    
    def __init__(self, env):
        self.env = env
    
    def act(self, state):
        "Returns a random action by sampling the action space."
        action = # TODO
        return action
    
    def update(self, state, action, reward, next_state):
        "Updates the agent using the transition (s, a, r, s')."
        pass
    
    def train(self, nb_episodes, render=False):
        "Runs the agent on the environment for nb_episodes. Returns the list of obtained returns."
        
        # List of returns
        returns = []

        # TODO
            
        return returns
```

The environment is passed to the constructor. `act(state)` should sample a random action. `update(state, action, reward, next_state)` does nothing for the random agent (`pass` is a Python command doing nothing), but we will implement it in the next exercises. 

`train(nb_episodes, render)` implements the interaction loop between the agent and the environment for a fixed number of episodes. It should return the list of obtained returns. `render` defines whether you print the state at each step or not.

**Q:** Implement the random agent and have it interact with the environment for a fixed number of episodes.

That's it! We now "only" need to define classes for all the sampling-based RL algorithms (MC, TD, deep RL) and we can interact with any environment with a single line!

## Mujoco and Atari environments

Note: both mujoco and atari environments will not work on colab. 

You may have to install non-Python packages on your computer, such as openGL. A lot of debugging in sight...

The environments should work under Linux and MacOS, but I am not sure about windows. 

### Mujoco

To install the mujoco environments of gymnasium, this should work:

```bash
pip install mujoco
pip install "gymnasium[mujoco]"
```

Interaction should work as usual. See all environments here: <https://gymnasium.farama.org/environments/mujoco/>

In [ ]:
envname = 'Walker2d-v4'
env = gym.make(envname, render_mode="rgb_array_list")
recorder = GymRecorder(env)

returns = random_interaction(env, 10, recorder)

video = "videos/" + envname + ".gif"
recorder.make_video(video)
ipython_display(video)

### Atari

The atari games are available as binary ROM files, which have to be downloaded separately. The AutoROM package can do that for you:  <https://github.com/Farama-Foundation/AutoROM>

```bash
pip install autorom
AutoROM --accept-license
```

You can then install the atari submodules of gym (in particular ale_py):

```bash
pip install "gymnasium[atari]"
```

Check out the list of Atari games here:  <https://gymnasium.farama.org/environments/atari/>

In [ ]:
import ale_py
gym.register_envs(ale_py)

envname= 'ALE/Breakout-v5'
env = gym.make(envname, render_mode="rgb_array_list")
recorder = GymRecorder(env)

returns = random_interaction(env, 10, recorder)

video = "videos/" + envname + ".gif"
recorder.make_video(video)
ipython_display(video)